In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.0 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
def token_healing(text):
    model_name = "gpt2"
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokens = text.split()
    healed_tokens = []
    for token in tokens:
        if tokenizer.unk_token in tokenizer.tokenize(token):
            input_text = f"The token {tokenizer.unk_token} is the input. Replace it with"
            input_ids = tokenizer.encode(input_text, return_tensors="pt")
            output_ids = model.generate(input_ids, max_length=20, num_return_sequences=1)
            healed_token = tokenizer.decode(output_ids[0], skip_special_tokens=True)
            healed_tokens.append(healed_token)
        else:
            healed_tokens.append(token)
    healed_text = ' '.join(healed_tokens)
    return healed_text
input_text = "Thiss is a testt."
healed_text = token_healing(input_text)
print(healed_text)

Thiss is a testt.


In [ ]:
import logging
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

def token_healing(text):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

    tokens = text.split()
    corrected_tokens = []

    for token in tokens:
        # Mask the token
        masked_text = text.replace(token, tokenizer.mask_token)

        # Tokenize the masked text
        input_ids = tokenizer.encode(masked_text, add_special_tokens=True, return_tensors="pt")

        # Generate predictions for the masked token
        with torch.no_grad():
            predictions = model(input_ids)[0]

        # Find the predicted token
        predicted_index = torch.argmax(predictions[0, -1, :]).item()
        predicted_token = tokenizer.decode(predicted_index)

        # Append the corrected token to the list
        corrected_tokens.append(predicted_token)

    healed_text = ' '.join(corrected_tokens)
    return healed_text

# Suppress the warning message
logging.getLogger("transformers").setLevel(logging.ERROR)

input_text = "Thiss is a testt."
healed_text = token_healing(input_text)
print(healed_text)


. . . .


In [ ]:
from transformers import pipeline

def token_healing(text):
    nlp_fill_mask = pipeline("fill-mask", model="bert-base-uncased")

    tokens = text.split()
    healed_tokens = []

    for token in tokens:
        if "[UNK]" in token:
            results = nlp_fill_mask(token.replace("[UNK]", ""))
            best_match = results[0]['token_str']
            healed_tokens.append(best_match)
        else:
            healed_tokens.append(token)

    healed_text = ' '.join(healed_tokens)
    return healed_text

input_text = "Thiss iss a testt ruun."
healed_text = token_healing(input_text)
print(healed_text)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Thiss iss a testt ruun.


In [ ]:
import logging
from transformers import pipeline

# Suppress the warning messages
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

def token_healing(text):
    nlp_fill_mask = pipeline("fill-mask", model="bert-base-uncased")

    tokens = text.split()
    healed_tokens = []

    for token in tokens:
        if "[UNK]" in token:
            results = nlp_fill_mask(token.replace("[UNK]", ""))
            best_match = results[0]['token_str']
            healed_tokens.append(best_match)
        else:
            healed_tokens.append(token)

    healed_text = ' '.join(healed_tokens)
    return healed_text

input_text = "Thiss is a testt ruun."
healed_text = token_healing(input_text)
print(healed_text)


Thiss is a testt ruun.


In [ ]:
from transformers import pipeline

def token_healing(text):
    nlp_ner = pipeline("ner", model="bert-base-uncased", tokenizer="bert-base-uncased")

    tokens = text.split()
    healed_tokens = []

    for token in tokens:
        if "[UNK]" in token:
            results = nlp_ner(token)
            best_match = results[0]['word']
            healed_tokens.append(best_match)
        else:
            healed_tokens.append(token)

    healed_text = ' '.join(healed_tokens)
    return healed_text

input_text = "Thiss is a testt."
healed_text = token_healing(input_text)
print(healed_text)


Thiss is a testt.
